In [1]:
import pandas as pd
import numpy
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('all_live.txt')

In [3]:
df.columns

Index([u'created_date', u'departure_day', u'departure_dt', u'duration', u'end_address', u'end_address_weather', u'route_type', u'start_address', u'start_address_weather'], dtype='object')

In [4]:
df.start_address_weather.unique()

array(['Snow', 'Rain', 'Clouds', 'Clear'], dtype=object)

In [5]:
df['weather_ind'] = df.start_address_weather.apply(lambda x: x if x in ['Rain','Snow'] else 'Clear')

In [6]:
print df[df.weather_ind == 'Rain'].shape
print df[df.weather_ind == 'Snow'].shape
print df[df.weather_ind == 'Clear'].shape

(87, 10)
(16, 10)
(168, 10)


In [7]:
df.groupby(['start_address', 'weather_ind']).duration.mean()

start_address                                       weather_ind
1001 Metro Center Blvd, Foster City, CA 94404, USA  Clear          2392.000000
                                                    Rain           1439.200000
104 8th St, Belgrade, MT 59714, USA                 Clear           985.666667
                                                    Rain           1074.000000
105 River St S, Santa Cruz, CA 95060, USA           Clear           194.800000
12 Corporate Blvd NE, Atlanta, GA 30329, USA        Clear           732.333333
                                                    Rain            771.000000
1470 Walnut St, Boulder, CO 80302, USA              Clear          1907.000000
1610 23rd Ave, San Francisco, CA 94122, USA         Clear          1499.000000
                                                    Rain           1357.000000
1714 Stockton St, San Francisco, CA 94133, USA      Clear          1641.000000
                                                    Rain           

In [28]:
df.groupby(['weather_ind']).duration.mean()

weather_ind
Clear          1362.238095
Rain           1389.011494
Snow           1081.437500
Name: duration, dtype: float64

## Group by address and weather logic

In [38]:
a = df.groupby(['start_address', 'weather_ind']).duration.mean()
a.shape

(62,)

In [39]:
clear_rec = {}
for ix in a.index:
    add, w = ix
    if w == 'Clear':
        clear_rec[add] = a[ix]

In [40]:
print len(clear_rec)

37


In [41]:
rain_rec = {}
for ix in a.index:
    add, w = ix
    if w == 'Rain':
        rain_rec[add] = a[ix]
print len(rain_rec)

19


In [42]:
snow_rec = {}
for ix in a.index:
    add, w = ix
    if w == 'Snow':
        snow_rec[add] = a[ix]
print len(snow_rec)

6


In [43]:
assert len(clear_rec)+len(rain_rec)+len(snow_rec) == a.shape[0]

In [50]:
comp_rain_clear = {}
count = 0
for add in rain_rec:
    if add in clear_rec:
        comp_rain_clear[add] = rain_rec[add] - clear_rec[add]

In [56]:
comp_rain_clear

{'1001 Metro Center Blvd, Foster City, CA 94404, USA': -952.79999999999995,
 '104 8th St, Belgrade, MT 59714, USA': 88.333333333333371,
 '12 Corporate Blvd NE, Atlanta, GA 30329, USA': 38.666666666666629,
 '1610 23rd Ave, San Francisco, CA 94122, USA': -142.0,
 '1714 Stockton St, San Francisco, CA 94133, USA': -397.40000000000009,
 '204 Ravens Crest Dr, Plainsboro Township, NJ 08536, USA': -20.111111111111086,
 '300 Cambridge Ave, Palo Alto, CA 94306, USA': -383.59999999999991,
 '448 E Main St, Bozeman, MT 59715, USA': 37.9375,
 '502 Carnegie Center, Princeton, NJ 08540, USA': -8.0795454545454959,
 '7700 Gateway Blvd, Newark, CA 94560, USA': -863.33333333333326,
 'Cambridge, Cambridgeshire CB3 0HB, UK': -49.333333333333258,
 'Cedar St & Edith St, Berkeley, CA 94703, USA': -1440.0,
 'Civic Center, San Francisco, CA, USA': -309.5,
 'E Main St & N Church Ave, Bozeman, MT 59715, USA': -25.5,
 'San Bruno BART-Bay 5, San Bruno, CA 94066, USA': -1105.1666666666667}

In [85]:
sec = 0
for k in comp_rain_clear:
    if 'CA' not in k:
        sec += comp_rain_clear[k]
#88.3+38.6-20+37.93-8.079-49.3-25
print "minutes delayed in rain: {} ({} seconds)".format(sec / 60, sec)

minutes delayed in rain: 1.03189183502 (61.913510101 seconds)


In [82]:
comp_snow_clear = {}
count = 0
for add in snow_rec:
    if add in clear_rec:
        comp_snow_clear[add] = snow_rec[add] - clear_rec[add]
print ["{}, {}".format(k, comp_snow_clear[k]) for k in comp_snow_clear]

['Cambridge, Cambridgeshire CB3 0HB, UK, -87.3333333333', '1801 W Bakerview Rd, Bellingham, WA 98226, USA, 4.5', 'Evergreen Way & Aspen Way, Nederland, CO 80466, USA, 26.0', 'San Bruno BART-Bay 5, San Bruno, CA 94066, USA, -1242.0']


In [84]:
comp_snow_rain = {}
count = 0
for add in snow_rec:
    if add in rain_rec:
        comp_snow_rain[add] = snow_rec[add] - rain_rec[add]
print ["{}, {}".format(k, comp_snow_rain[k]) for k in comp_snow_rain]

['Cambridge, Cambridgeshire CB3 0HB, UK, -38.0', 'Cascade St & Sanders Ave, Bozeman, MT 59718, USA, 76.5', 'San Bruno BART-Bay 5, San Bruno, CA 94066, USA, -136.833333333']
